In [ ]:
# !git clone https://github.com/hccho2/Tacotron2-Wavenet-Korean-TTS.git

In [ ]:
# import os
# os.chdir('/content/drive/My Drive/Project/TTS_Transfer')
# os.getcwd()
# !git clone https://github.com/TensorSpeech/TensorFlowTTS.git
# os.chdir("TensorFlowTTS")
# !pip install .
# os.chdir("..")

# import sys
# sys.path.append("TensorFlowTTS/")

In [ ]:
import os
os.chdir('/content/drive/My Drive/Project/TTS_audiobook/TensorFlowTTS')
print(os.getcwd())
! pip install TensorFlowTTS

import sys
sys.path.append("TensorFlowTTS/")

In [ ]:
import os
os.chdir('/content/drive/My Drive/Project/TTS_audiobook/TensorFlowTTS')
os.getcwd()
os.listdir()

In [ ]:
import tensorflow as tf

import pandas as pd
import yaml
import numpy as np
import matplotlib.pyplot as plt

import IPython.display as ipd

from tensorflow_tts.inference import AutoConfig
from tensorflow_tts.inference import TFAutoModel
from tensorflow_tts.inference import AutoProcessor

import nltk
nltk.download('punkt')

In [ ]:
# 데이터 전처리 
!tensorflow-tts-preprocess --rootdir ./kss --outdir ./dump_yin_40 --config ./preprocess/kss_preprocess.yaml --dataset kss

In [ ]:
# 데이터 Normailizing 
!tensorflow-tts-normalize --rootdir ./dump_yin_40 --outdir ./dump_yin_40 --config ./preprocess/kss_preprocess.yaml --dataset kss

In [ ]:
# 타코트론 학습 
!CUDA_VISIBLE_DEVICES=0 python ./ttsexamples/tacotron2/train_tacotron2.py\
  --train-dir ./dump_book/train/ \
  --dev-dir ./dump_book/valid/ \
  --outdir ./ttsexamples/tacotron2/exp/train.tacotron2.v2/ \
  --config ./ttsexamples/tacotron2/conf/tacotron2.kss.v1.yaml \
  --use-norm 1 \
  --mixed_precision 0 \
  --resume ./ttsexamples/multiband_melgan/exp/train.multiband_melgan.v2/checkpoints/ckpt-3500
# --pretrained ./ttsexamples/tacotron2/exp/train.tacotron2.v1/checkpoints/model-88000.h5

In [ ]:
# melgan 학습 
!CUDA_VISIBLE_DEVICES=0 python ./ttsexamples/multiband_melgan/train_multiband_melgan.py\
  --train-dir ./dump_moon/train/ \
  --dev-dir ./dump_moon/valid/ \
  --outdir ./ttsexamples/multiband_melgan/exp/train.multiband_melgan.v1/ \
  --config ./ttsexamples/multiband_melgan/conf/multiband_melgan.v1.yaml \
  --use-norm 1 \
  --generator_mixed_precision 1 \
  --resume ./ttsexamples/multiband_melgan/exp/train.multiband_melgan.v1/checkpoints/ckpt-240000

## Inference

In [ ]:
tacotron2_config = AutoConfig.from_pretrained('/content/drive/My Drive/Project/TTS_audiobook/TensorFlowTTS/ttsexamples/tacotron2/conf/tacotron2.kss.v1.yaml')
tacotron2 = TFAutoModel.from_pretrained(
    config=tacotron2_config,
    pretrained_path="/content/drive/My Drive/Project/TTS_audiobook/TensorFlowTTS/ttsexamples/tacotron2/exp/train.tacotron2.v4/checkpoints/model-86000.h5",
    training=False, 
    name="tacotron2"
)

#  pretrained_path="/content/drive/My Drive/Project/TTS_Transfer/TensorFlowTTS/ttsexamples/tacotron2/exp/train.tacotron2.v1/checkpoints/model-12000.h5

In [ ]:
mb_melgan_config = AutoConfig.from_pretrained('/content/drive/My Drive/Project/TTS/TensorFlowTTS/ttsexamples/multiband_melgan/conf/multiband_melgan.v1.yaml')
mb_melgan = TFAutoModel.from_pretrained(
    config=mb_melgan_config,
    pretrained_path="/content/drive/My Drive/Project/TTS_YIN/TensorFlowTTS/ttsexamples/multiband_melgan/exp/train.multiband_melgan.v1/checkpoints/generator-360000.h5",
    name="mb_melgan"
)

In [ ]:
processor = AutoProcessor.from_pretrained(pretrained_path="/content/drive/My Drive/Project/TTS_audiobook/TensorFlowTTS/dump_yu_new/kss_mapper.json")

In [ ]:
processor

In [ ]:
def do_synthesis(input_text, text2mel_model, vocoder_model, text2mel_name, vocoder_name):
  input_ids = processor.text_to_sequence(input_text)

  # text2mel part
  if text2mel_name == "TACOTRON":
    _, mel_outputs, stop_token_prediction, alignment_history = text2mel_model.inference(
        tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
        tf.convert_to_tensor([len(input_ids)], tf.int32),
        tf.convert_to_tensor([0], dtype=tf.int32)
    )
  elif text2mel_name == "FASTSPEECH2":
    mel_before, mel_outputs, duration_outputs, _, _ = text2mel_model.inference(
        tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
        speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
        speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
        f0_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
        energy_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
    )
  else:
    raise ValueError("Only TACOTRON, FASTSPEECH2 are supported on text2mel_name")

  # vocoder part
  if vocoder_name == "MB-MELGAN":
    audio = vocoder_model.inference(mel_outputs)[0, :, 0]
  else:
    raise ValueError("Only MB_MELGAN are supported on vocoder_name")

  if text2mel_name == "TACOTRON":
    return mel_outputs.numpy(), alignment_history.numpy(), audio.numpy()
  else:
    return mel_outputs.numpy(), audio.numpy()

def visualize_attention(alignment_history):
  import matplotlib.pyplot as plt

  fig = plt.figure(figsize=(8, 6))
  ax = fig.add_subplot(111)
  ax.set_title(f'Alignment steps')
  im = ax.imshow(
      alignment_history,
      aspect='auto',
      origin='lower',
      interpolation='none')
  fig.colorbar(im, ax=ax)
  xlabel = 'Decoder timestep'
  plt.xlabel(xlabel)
  plt.ylabel('Encoder timestep')
  plt.tight_layout()
  plt.show()
  plt.close()

def visualize_mel_spectrogram(mels):
  mels = tf.reshape(mels, [-1, 80]).numpy()
  fig = plt.figure(figsize=(10, 8))
  ax1 = fig.add_subplot(311)
  ax1.set_title(f'Predicted Mel-after-Spectrogram')
  im = ax1.imshow(np.rot90(mels), aspect='auto', interpolation='none')
  fig.colorbar(mappable=im, shrink=0.65, orientation='horizontal', ax=ax1)
  plt.show()
  plt.close()

In [ ]:
input_text = "뭐하는 거니 너 지금"

In [ ]:
tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0)

In [ ]:
tacotron2.inference(tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0), tf.convert_to_tensor([len(input_ids)], tf.int32), tf.convert_to_tensor([0], dtype=tf.int32))

In [ ]:
mels, alignment_history, audios = do_synthesis(input_text, tacotron2, mb_melgan, "TACOTRON", "MB-MELGAN")
visualize_attention(alignment_history[0])
visualize_mel_spectrogram(mels[0])
ipd.Audio(audios, rate=22050)